In [4]:
# code by Tae Hwan Jung @graykode
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split() #先通过空格分词
print(word_list)
word_list = list(set(word_list))#先去重再构建成列表
word_dict = {w: i for i, w in enumerate(word_list)} #通过文字索引编号
number_dict = {i: w for i, w in enumerate(word_list)}#通过编号索引文字
n_class = len(word_dict) # number of Vocabulary 
print('word_list:',word_list)
print('word_dict:',word_dict)
print('number_dict',number_dict)

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']
word_list: ['love', 'milk', 'like', 'hate', 'dog', 'coffee', 'i']
word_dict: {'love': 0, 'milk': 1, 'like': 2, 'hate': 3, 'dog': 4, 'coffee': 5, 'i': 6}
number_dict {0: 'love', 1: 'milk', 2: 'like', 3: 'hate', 4: 'dog', 5: 'coffee', 6: 'i'}


In [5]:

# NNLM Parameter
n_step = 2 # n-1 in paper
n_hidden = 2 # h in paper
m = 2 # m in paper
#建立batch
def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]] #前面的词作为input
        target = word_dict[word[-1]]#最后一个词作为target

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch

# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m) #n_class是词表大小,m是目标embedding的维数
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype)) #建立一个n_step * m,n_hidden的可训练的参数层
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype)) #建立一个n_hidden的参数层
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))#n_hidden, n_class
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))

    def forward(self, X):
        X = self.C(X) #先embedding 
        X = X.view(-1, n_step * m) # [batch_size, n_step * n_class] #展开为输入层一致的格式
        tanh = torch.tanh(self.d + torch.mm(X, self.H)) # [batch_size, n_hidden]#隐藏层
        output = self.b + torch.mm(X, self.W) + torch.mm(tanh, self.U) # [batch_size, n_class]#输出层
        return output

model = NNLM()

criterion = nn.CrossEntropyLoss() #交叉熵作为损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001) #Adam优化器

input_batch, target_batch = make_batch(sentences) #输入输出
input_batch = Variable(torch.LongTensor(input_batch)) #包装为Variable
target_batch = Variable(torch.LongTensor(target_batch))

# Training
for epoch in range(5000):

    optimizer.zero_grad() #清零梯度
    output = model(input_batch)#预测

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)#损失计算
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()#反向传播
    optimizer.step()#下一步

# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

# Test
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])



Epoch: 1000 cost = 0.204264
Epoch: 2000 cost = 0.032313
Epoch: 3000 cost = 0.009893
Epoch: 4000 cost = 0.004144
Epoch: 5000 cost = 0.002002
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
